(c) European Space Agency (ESA) Licensed under ESA Software Community Licence Permissive (Type 3) – v2.4

Example on how to query the ASCEND Catalogue and download BIOMASS QL data.
This requires a token generated with a dedicated account. 


In [ ]:
from pystac_client import Client 
from pystac import Collection
from tqdm import tqdm
import pandas as pd 
import requests
import json
import os

In [ ]:

URL_LANDING_PAGE =  'https://catalog.maap.eo.esa.int/catalogue/'  # Operational 

In [ ]:
api = Client.open(URL_LANDING_PAGE) 
# show as a dictionary
#api.to_dict()



Filter - allows to search for different metadata parameters.  e.g. "modificationDate" as hereafter:
#filter="modificationDate > TIMESTAMP('2025-01-10T09:54:34.651Z')"

Datetime represents the coverage of the data. None can be used for start and finish to indicate unbounded queries. 
#datetime=['2017-01-01T00:00:00Z', None]

BIO_COLLECTIONS - Represents the data to be queried; is can be a list.

In [ ]:
#BIO_COLLECTIONS=['BiomassLevel0IOC']
#BIO_COLLECTIONS=['BiomassAuxIOC']
#BIO_COLLECTIONS=['BiomassSimulated']
BIO_COLLECTIONS=['BiomassLevel1aIOC']
results = api.search(
    method="GET",
    max_items = 500,
    collections=BIO_COLLECTIONS,
    #filter="modificationDate > TIMESTAMP('2025-05-23T00:00:00.000Z') and productType='S1_RAW__0S'"
    filter="productType='S1_SCS__1S'",
   datetime=['2025-09-29T00:00:00Z', '2025-09-29T00:12:00Z'] 
)

print(f"{results.matched()} items found.")

Using Pandas dataframes for ease of use, extracting the urls to download.

download_url is the full zip
quicklook_url is the png quick look
kmz_url is the kmz overview that requires the png.

In [ ]:
data = results.item_collection_as_dict()
#df=pd.json_normalize(data,record_path=['features'])
df = pd.json_normalize(data, record_path=['features'])[["id","collection","properties.product:type","assets.product.href","assets.product.file:local_path","assets.quicklook.href","assets.quicklook.title","assets.quicklook_1.href","assets.quicklook_1.title","properties.updated"]]
df.rename(columns={'properties.product:type': 'product_type', 'assets.product.href': 'download_url', 'assets.product.file:local_path': 'product_name','properties.updated': 'last_modified','assets.quicklook.href': 'quicklook_url','assets.quicklook.title': 'quicklook_name','assets.quicklook_1.href': 'kmz_url','assets.quicklook_1.title': 'kmz_name'}, inplace=True)
df.sort_values(by = 'id', ascending=True, inplace=True)

df

This is the data download part, it requires a token (using the functional user) to be saved locally. 

Token needs to be obtained directly for the functional user - using the sample script or directly to the server.

__TOKEN is most probably not valid, the valid can be used if the user obtain the token from the url and doesn't use a token.txt file.
Users can generate and retrieve the token here:
➡️ https://portal.maap.eo.esa.int/ini/services/auth/token/

In [ ]:
import pathlib
if pathlib.Path("token.txt").exists():
  with open("token.txt","rt") as f:
    token = f.read().strip().replace("\n","")
    print("Got token")
else:
  token=__TOKEN

def download_file_with_bearer_token(url, token, file_path, disable_bar=False):
  """
  Downloads a file from a given URL using a Bearer token.
  """

  try:
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes
    file_size = int(response.headers.get('content-length', 0))

    chunk_size = 8 * 1024 * 1024 # Byes - 1MiB
    with open(file_path, "wb") as f, tqdm(
        desc=file_path,
        total=file_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
        disable=disable_bar,
      ) as bar:
      for chunk in response.iter_content(chunk_size=chunk_size):
        read_size=f.write(chunk)
        bar.update(read_size)

    if (disable_bar): 
      print(f"File downloaded successfully to {file_path}")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")

# # test one...
# my_test_url="https://catalog.maap.eo.esa.int/data/zipper/biomass-pdgs-01/BiomassLevel1aIOC/2025/06/06/BIO_S1_SCS__1S_20250606T171259_20250606T171320_C_G___M___C___T____F136_01_D9UITH/BIO_S1_SCS__1S_20250606T171259_20250606T171320_C_G___M___C___T____F136_01_D9UITH"
# my_test_name="BIO_S1_SCS__1S_20250606T171259_20250606T171320_C_G___M___C___T____F136_01_D9UITH.ZIP"
# download_file_with_bearer_token(my_test_url, token, my_test_name)

One has to be very carefull below, it can download a lot of data if the df is huge! 

It can and should commented out if one just wants the QL (next cells).

In [ ]:
# Download products
# For full dataframe remove ".head()." call
# for i in df.head(4).index:
#     download_file_with_bearer_token(df.loc[i]['download_url'], token, df.loc[i]['product_name'])

download_file_with_bearer_token(df.loc[0]['download_url'], token, df.loc[0]['product_name'])

The following will download all the QL files for the products in the df, make sure that the query matches your needs before running this one.

In [ ]:
# Download quicklooks and kmz
# For full dataframe remove ".head()." call... df.head(2).index or df.index
for i in df.index:
    filename = os.path.splitext(df.loc[i]['kmz_name'])[0] 
    download_file_with_bearer_token(df.loc[i]['quicklook_url'], token, df.loc[i]['quicklook_name'])
    download_file_with_bearer_token(df.loc[i]['kmz_url'], token, df.loc[i]['kmz_name'])
    index = filename.find("_map")
    filename = filename[:index] + "_ql.png"
    os.rename(df.loc[i]['quicklook_name'], filename)
    
    